# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.


In [2]:
stocks = pd.read_csv('/Users/sagarprasad/Downloads/algorithmic-trading-python-master/starter_files/sp_500_stocks.csv')

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using.

In [4]:
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# print(data.status_code != 200) "it can be used for comments
# print(data['symbol'])
data

{'avgTotalVolume': 56888460,
 'calculationPrice': 'close',
 'change': 0.49,
 'changePercent': 0.00282,
 'close': 174.49,
 'closeSource': 'official',
 'closeTime': 1692388800378,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 174.49,
 'delayedPriceTime': 1692388799923,
 'extendedChange': 0.21,
 'extendedChangePercent': 0.0012,
 'extendedPrice': 174.7,
 'extendedPriceTime': 1692403196716,
 'high': 175.1,
 'highSource': '15 minute delayed price',
 'highTime': 1692388799923,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 174.54,
 'iexBidSize': 100,
 'iexClose': 174.49,
 'iexCloseTime': 1692388797614,
 'iexLastUpdated': 1692388797614,
 'iexMarketPercent': 0.015124807612614564,
 'iexOpen': 172.06,
 'iexOpenTime': 1692365400253,
 'iexRealtimePrice': 174.49,
 'iexRealtimeSize': 200,
 'iexVolume': 925217,
 'lastTradeTime': 1692388799923,
 'latestPrice': 174.49,
 'latestSource': 'Close',
 'latestTime': 'August 18, 2023',
 'latestUpdate': 1692388800378,
 'latestVolume': 6

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
# print(market_cap/1000000000000)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

/var/folders/fz/_9lzp1vd2r9bqxn8xcqqvnrm0000gn/T/ipykernel_28962/3975751442.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,174.49,2728017141680,N/A


In [9]:
import warnings
warnings.filterwarnings("ignore")

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:75]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stocks,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
        index = my_columns),
        ignore_index = True
        )

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,Ticker 0 A 1 AAL 2 AAP 3 ...,119.25,35223566297,N/A
1,Ticker 0 A 1 AAL 2 AAP 3 ...,15.04,9826571037,N/A
2,Ticker 0 A 1 AAL 2 AAP 3 ...,69.83,4150964185,N/A
3,Ticker 0 A 1 AAL 2 AAP 3 ...,174.49,2728017141680,N/A
4,Ticker 0 A 1 AAL 2 AAP 3 ...,150.14,265004108535,N/A
...,...,...,...,...
70,Ticker 0 A 1 AAL 2 AAP 3 ...,179.00,21142918298,N/A
71,Ticker 0 A 1 AAL 2 AAP 3 ...,352.56,765173602340,N/A
72,Ticker 0 A 1 AAL 2 AAP 3 ...,50.19,73489365570,N/A
73,Ticker 0 A 1 AAL 2 AAP 3 ...,39.64,9317898113,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
def chunks(lst, n):
    """yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i +n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else :
            final_dataframe = final_dataframe.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.25,35223566297,N/A
1,AAL,15.04,9826571037,N/A
2,AAP,69.83,4150964185,N/A
3,AAPL,174.49,2728017141680,N/A
4,ABBV,150.14,265004108535,N/A
...,...,...,...,...
496,YUM,129.12,36180880603,N/A
497,ZBH,119.18,24904345490,N/A
498,ZBRA,271.98,13963008241,N/A
499,ZION,34.73,5145080469,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [14]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000


In [15]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
# print(final_dataframe[:2])
# number_of_shares = 39.999 
# print(math.floor(number_of_shares))

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

### Initializing our XlsxWriter Object

In [16]:
writer = pd.ExcelWriter('recommended trades.xlsx',engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [18]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format )
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format )
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format )
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format )

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [19]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, string_format)
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [20]:
writer.save()